# Data import

In [1]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("dogcdt/synapse")
print("Path to dataset files:", path)

100%|██████████| 607M/607M [00:29<00:00, 21.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dogcdt/synapse/versions/1


# losses and metrics

In [2]:
pip install medpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 11.2 MB/s eta 0:00:00
  Created wheel for medpy: filename=MedPy-0.5.2-py3-none-any.whl size=224710 sha256=89ce6602b3067aa0b0c56835c9982ae11f337cfa0dfbbd38e9ad0a6d933881bc
  Stored in directory: /root/.cache/pip/wheels/89/5a/f8/b3def53b9c2133d2f8698ea2173bb5df63bd8e761ce8e9aec9
Successfully built medpy


In [4]:
import numpy as np
import torch
from medpy import metric
from scipy.ndimage import zoom
import torch.nn as nn
import SimpleITK as sitk

## metrics
def calculate_metric_percase(pred, gt):
    pred[pred > 0] = 1
    gt[gt > 0] = 1
    if pred.sum() > 0 and gt.sum()>0:
        dice = metric.binary.dc(pred, gt)
        hd95 = metric.binary.hd95(pred, gt)
        jac = metric.binary.jc(pred, gt)
        return dice, jac, hd95
    elif pred.sum() > 0 and gt.sum()==0:
        return 1, 1, 0
    else:
        return 0, 0, 0

def multilabel_metric(pred, gt, num_classes):
    gt = gt.squeeze(0)
    metric_list = []
    for i in range(1, num_classes):
        metric_list.append(calculate_metric_percase(pred == i, gt == i))
    return metric_list  # list, lenth=num_classes-1, 每个元素含有(x,y,z)

# losses
class BinaryDiceLoss(nn.Module):
    def init(self):
        super(BinaryDiceLoss, self).init()

    def forward(self, input, targets):
        # 获取每个批次的大小 N
        N = targets.size()[0]
        # 平滑变量
        smooth = 1
        # 将宽高 reshape 到同一纬度
        input_flat = input.view(N, -1)
        targets_flat = targets.view(N, -1)
        # 计算交集
        intersection = input_flat * targets_flat
        N_dice_eff = (2 * intersection.sum(1) + smooth) / (input_flat.sum(1) + targets_flat.sum(1) + smooth)
        # 计算一个批次中平均每张图的损失
        loss = 1 - N_dice_eff.sum() / N
        return loss


class MultiClassDiceLoss(nn.Module):
    def init(self, weight=None, ignore_index=None, **kwargs):
        super(MultiClassDiceLoss, self).init()
        self.weight = weight
        self.ignore_index = ignore_index
        self.kwargs = kwargs
    def forward(self, input, target):
        """
        input tesor of shape = (N, C, H, W)
        target tensor of shape = (N, H, W)
        """
        # 先将 target 进行 one-hot 处理，转换为 (N, C, H, W)
        # target[target==255.] = 0.
        nclass = input.shape[1]
        target = F.one_hot(target.long(), nclass)
        target = target.reshape(input.shape[0],input.shape[1],input.shape[2],-1)

        assert input.shape == target.shape, "predict & target shape do not match"
        binaryDiceLoss = BinaryDiceLoss()
        total_loss = 0
        # 归一化输出
        logits = F.softmax(input, dim=1)
        C = target.shape[1]
        # 遍历 channel，得到每个类别的二分类 DiceLoss
        for i in range(C):
            dice_loss = binaryDiceLoss(logits[:, i], target[:, i])
            total_loss += dice_loss
            # 每个类别的平均 dice_loss
        return total_loss / C

# dataset manipulation

In [5]:
import random
import h5py
import numpy as np
import torch
from scipy import ndimage
from scipy.ndimage import zoom
from torch.utils.data import Dataset
from pathlib import Path


# -------------------------
# Augmentations
# -------------------------
def random_rot_flip(image, label):
    k = np.random.randint(0, 4)
    image = np.rot90(image, k)
    label = np.rot90(label, k)
    axis = np.random.randint(0, 2)
    image = np.flip(image, axis=axis).copy()
    label = np.flip(label, axis=axis).copy()
    return image, label


def random_rotate(image, label):
    angle = np.random.randint(-20, 20)
    image = ndimage.rotate(image, angle, order=0, reshape=False)
    label = ndimage.rotate(label, angle, order=0, reshape=False)
    return image, label


class RandomGenerator(object):
    def __init__(self, output_size):
        self.output_size = output_size  # (H, W)

    def __call__(self, sample):
        image, label = sample["image"], sample["label"]

        if random.random() > 0.5:
            image, label = random_rot_flip(image, label)
        elif random.random() > 0.5:
            image, label = random_rotate(image, label)

        x, y = image.shape
        if (x != self.output_size[0]) or (y != self.output_size[1]):
            image = zoom(image, (self.output_size[0] / x, self.output_size[1] / y), order=3)
            label = zoom(label, (self.output_size[0] / x, self.output_size[1] / y), order=0)

        image = torch.from_numpy(image.astype(np.float32)).unsqueeze(0)  # (1, H, W)
        label = torch.from_numpy(label.astype(np.int64))                 # (H, W)
        return {"image": image, "label": label}


# -------------------------
# Dataset
# -------------------------
class Synapse_dataset(Dataset):
    """
    Your structure:

    /root/.cache/kagglehub/datasets/dogcdt/synapse/versions/1/Synapse/
        train_npz/
        test_vol_h5/   (or whatever your test folder is)
    /root/lists_synapse/
        train.txt
        test_vol.txt

    Train slices: <name>.npz with keys (image, label)
    Test volumes: <name>.npy.h5 OR <name>.h5 OR <name>.n5 with datasets (image, label)
    """

    def __init__(
        self,
        versions_root="/root/.cache/kagglehub/datasets/dogcdt/synapse/versions/1",
        split="train",
        transform=None,
        list_dir="/root/lists_synapse",
        data_subdir="Synapse",          # <-- IMPORTANT: your extra folder level
        train_folder="train_npz",
        test_folder="test_vol_h5",
        train_list="train.txt",
        test_list="test_vol.txt",
        verbose=True,
    ):
        self.transform = transform
        self.split = split.lower().strip()

        versions_root = Path(versions_root).expanduser().resolve()
        self.data_root = (versions_root / data_subdir).resolve()  # <-- /.../versions/1/Synapse

        list_dir = Path(list_dir).expanduser()
        self.list_dir = list_dir.resolve() if list_dir.is_absolute() else (self.data_root / list_dir).resolve()

        self.train_dir = (self.data_root / train_folder).resolve()
        self.test_dir = (self.data_root / test_folder).resolve()

        list_file = self.list_dir / (train_list if self.split == "train" else test_list)

        if verbose:
            print("VERSIONS ROOT:", versions_root)
            print("DATA ROOT    :", self.data_root)
            print("LIST DIR     :", self.list_dir)
            print("LIST FILE    :", list_file)
            print("TRAIN DIR    :", self.train_dir)
            print("TEST  DIR    :", self.test_dir)

        if not list_file.is_file():
            raise FileNotFoundError(f"Missing list file: {list_file}")

        self.sample_list = list_file.read_text().splitlines()

        if self.split == "train" and not self.train_dir.is_dir():
            raise FileNotFoundError(f"Missing folder: {self.train_dir}")
        if self.split != "train" and not self.test_dir.is_dir():
            raise FileNotFoundError(f"Missing folder: {self.test_dir}")

    def __len__(self):
        return len(self.sample_list)

    def _resolve_train_path(self, name: str) -> Path:
        return self.train_dir / f"{name}.npz"

    def _resolve_test_path(self, name: str) -> Path:
        candidates = [
            self.test_dir / f"{name}.npy.h5",
            self.test_dir / f"{name}.h5",
            self.test_dir / f"{name}.n5",
        ]
        for p in candidates:
            if p.is_file():
                return p
        return candidates[0]

    def show_paths(self, max_items=20):
        n = min(len(self.sample_list), max_items)
        print(f"Showing {n}/{len(self.sample_list)} paths for split='{self.split}'")
        for i in range(n):
            name = self.sample_list[i].strip()
            p = self._resolve_train_path(name) if self.split == "train" else self._resolve_test_path(name)
            print(p, "| exists =", p.exists())

    def __getitem__(self, idx):
        name = self.sample_list[idx].strip()

        if self.split == "train":
            data_path = self._resolve_train_path(name)
            if not data_path.is_file():
                raise FileNotFoundError(f"Missing train npz: {data_path}")

            data = np.load(str(data_path))
            image, label = data["image"], data["label"]

        else:
            h5_path = self._resolve_test_path(name)
            if not h5_path.is_file():
                raise FileNotFoundError(
                    "Missing test volume. Tried: "
                    f"{self.test_dir / (name + '.npy.h5')}, "
                    f"{self.test_dir / (name + '.h5')}, "
                    f"{self.test_dir / (name + '.n5')}"
                )

            with h5py.File(str(h5_path), "r") as data:
                image, label = data["image"][:], data["label"][:]

        sample = {"image": image, "label": label}
        if self.transform is not None:
            sample = self.transform(sample)

        sample["case_name"] = name
        return sample


# -------------------------
# Quick local test
# -------------------------
if __name__ == "__main__":
    versions_root = "/root/.cache/kagglehub/datasets/dogcdt/synapse/versions/1"
    list_dir = "/root/lists_synapse"

    ds_train = Synapse_dataset(versions_root=versions_root, split="train", transform=None, list_dir=list_dir)
    ds_train.show_paths(max_items=30)

    ds_test = Synapse_dataset(versions_root=versions_root, split="test", transform=None, list_dir=list_dir)
    ds_test.show_paths(max_items=10)

In [6]:
"""
https://github.com/Minerva-J/Pytorch-Segmentation-multi-models/blob/master/models/AttentionUnet/AttUnet.py
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

class conv_block(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(conv_block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class up_conv(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(up_conv, self).__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.up(x)
        return x

class Attention_block(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(Attention_block, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)

        return x * psi


class U_Net(nn.Module):
    def __init__(self, img_ch=3, num_class=9):
        super(U_Net, self).__init__()

        self.Maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(ch_in=img_ch, ch_out=64)
        self.Conv2 = conv_block(ch_in=64, ch_out=128)
        self.Conv3 = conv_block(ch_in=128, ch_out=256)
        self.Conv4 = conv_block(ch_in=256, ch_out=512)
        self.Conv5 = conv_block(ch_in=512, ch_out=1024)

        self.Up5 = up_conv(ch_in=1024, ch_out=512)
        self.Up_conv5 = conv_block(ch_in=1024, ch_out=512)

        self.Up4 = up_conv(ch_in=512, ch_out=256)
        self.Up_conv4 = conv_block(ch_in=512, ch_out=256)

        self.Up3 = up_conv(ch_in=256, ch_out=128)
        self.Up_conv3 = conv_block(ch_in=256, ch_out=128)

        self.Up2 = up_conv(ch_in=128, ch_out=64)
        self.Up_conv2 = conv_block(ch_in=128, ch_out=64)

        self.Conv_1x1 = nn.Conv2d(64, num_class, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        # encoding path
        x1 = self.Conv1(x)

        x2 = self.Maxpool(x1)
        x2 = self.Conv2(x2)

        x3 = self.Maxpool(x2)
        x3 = self.Conv3(x3)

        x4 = self.Maxpool(x3)
        x4 = self.Conv4(x4)

        x5 = self.Maxpool(x4)
        x5 = self.Conv5(x5)

        # decoding + concat path
        d5 = self.Up5(x5)
        d5 = torch.cat((x4, d5), dim=1)

        d5 = self.Up_conv5(d5)

        d4 = self.Up4(d5)
        d4 = torch.cat((x3, d4), dim=1)
        d4 = self.Up_conv4(d4)

        d3 = self.Up3(d4)
        d3 = torch.cat((x2, d3), dim=1)
        d3 = self.Up_conv3(d3)

        d2 = self.Up2(d3)
        d2 = torch.cat((x1, d2), dim=1)
        d2 = self.Up_conv2(d2)

        # d1 = self.Conv_1x1(d2)
        d1 = self.Conv_1x1(d2).squeeze(1)

        return d1

# Main

In [ ]:
import os
import sys
import random
import argparse
import datetime
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from medpy import metric as medpy_metric
from google.colab import drive


# ============================================================
#                 Metrics (MedPy)
# ============================================================

def calculate_metric_percase(pred, gt):
    pred = pred.astype(np.uint8)
    gt = gt.astype(np.uint8)
    pred[pred > 0] = 1
    gt[gt > 0] = 1

    if gt.sum() == 0:
        return np.nan, np.nan, np.nan

    if pred.sum() == 0:
        return 0.0, 0.0, np.nan

    dice = medpy_metric.binary.dc(pred, gt)
    jac = medpy_metric.binary.jc(pred, gt)
    hd95 = medpy_metric.binary.hd95(pred, gt)
    return float(dice), float(jac), float(hd95)


def multilabel_metric(pred_2d, gt_2d, num_classes):
    metric_list = []
    for i in range(1, num_classes):
        metric_list.append(calculate_metric_percase(pred_2d == i, gt_2d == i))
    return metric_list


def init_running_metrics(num_classes: int):
    return {
        "dice_sum": {c: 0.0 for c in range(1, num_classes)},
        "jac_sum":  {c: 0.0 for c in range(1, num_classes)},
        "hd95_sum": {c: 0.0 for c in range(1, num_classes)},
        "count":    {c: 0   for c in range(1, num_classes)},
    }


def update_running_metrics(running, pred_2d, gt_2d, num_classes: int):
    mlist = multilabel_metric(pred_2d, gt_2d, num_classes)
    for cls_idx, (dice, jac, hd95) in enumerate(mlist, start=1):
        if not np.isnan(dice):
            running["dice_sum"][cls_idx] += float(dice)
        if not np.isnan(jac):
            running["jac_sum"][cls_idx] += float(jac)
        if not np.isnan(hd95):
            running["hd95_sum"][cls_idx] += float(hd95)
        if (not np.isnan(dice)) or (not np.isnan(jac)) or (not np.isnan(hd95)):
            running["count"][cls_idx] += 1
    return running


def finalize_metrics(running, num_classes: int):
    per_class = {}
    dice_vals, jac_vals, hd95_vals = [], [], []

    for c in range(1, num_classes):
        cnt = running["count"][c]
        if cnt > 0:
            d = running["dice_sum"][c] / cnt
            j = running["jac_sum"][c] / cnt
            h = running["hd95_sum"][c] / cnt
        else:
            d, j, h = np.nan, np.nan, np.nan

        per_class[c] = (float(d) if not np.isnan(d) else np.nan,
                        float(j) if not np.isnan(j) else np.nan,
                        float(h) if not np.isnan(h) else np.nan)

        dice_vals.append(d)
        jac_vals.append(j)
        hd95_vals.append(h)

    macro_dice = float(np.nanmean(dice_vals)) if len(dice_vals) else np.nan
    macro_jac  = float(np.nanmean(jac_vals))  if len(jac_vals)  else np.nan
    macro_hd95 = float(np.nanmean(hd95_vals)) if len(hd95_vals) else np.nan

    return per_class, (macro_dice, macro_jac, macro_hd95)


# ============================================================
#                 Losses (Dice excludes background)
# ============================================================

class BinaryDiceLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input, targets):
        N = targets.size(0)
        smooth = 1.0
        input_flat = input.view(N, -1)
        targets_flat = targets.view(N, -1)
        inter = input_flat * targets_flat
        dice_eff = (2 * inter.sum(1) + smooth) / (input_flat.sum(1) + targets_flat.sum(1) + smooth)
        return 1 - dice_eff.sum() / N


class MultiClassDiceLoss(nn.Module):
    def __init__(self, class_weights=None, include_background=False):
        super().__init__()
        self.class_weights = class_weights
        self.include_background = include_background
        self.bdl = BinaryDiceLoss()

    def forward(self, logits, target):
        C = logits.shape[1]
        target_oh = F.one_hot(target.long(), C).permute(0, 3, 1, 2).contiguous()
        probs = F.softmax(logits, dim=1)

        start_c = 0 if self.include_background else 1
        losses = []
        for c in range(start_c, C):
            losses.append(self.bdl(probs[:, c], target_oh[:, c]))

        losses = torch.stack(losses, dim=0)

        if self.class_weights is not None:
            w = self.class_weights.to(losses.device)
            if not self.include_background:
                w = w[1:]
            w = w / (w.mean().clamp_min(1e-12))
            return (losses * w).mean()

        return losses.mean()


# ============================================================
#                 Drive logging + CSV
# ============================================================

class TeeLogger:
    def __init__(self, filename):
        self.terminal = sys.stdout
        self.log = open(filename, "a", encoding="utf8")

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)

    def flush(self):
        self.terminal.flush()
        self.log.flush()
        try:
            os.fsync(self.log.fileno())
        except Exception:
            pass


def setup_drive_logging(exp_dir):
    logs_dir = os.path.join(exp_dir, "logs")
    os.makedirs(logs_dir, exist_ok=True)
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    log_path = os.path.join(logs_dir, f"train_{ts}.log")
    sys.stdout = TeeLogger(log_path)
    return log_path


def append_csv(csv_path: str, header: str, line: str):
    exists = os.path.isfile(csv_path)
    with open(csv_path, "a", encoding="utf8") as f:
        if not exists:
            f.write(header)
        f.write(line)
        f.flush()
        try:
            os.fsync(f.fileno())
        except Exception:
            pass


# ============================================================
#                 Option C: estimate weights
# ============================================================

def estimate_ce_weights_from_loader(train_loader, num_classes, device,
                                   max_batches=200, clamp_min=0.1, clamp_max=10.0):
    counts = torch.zeros(num_classes, dtype=torch.float64)
    for k, s in enumerate(train_loader):
        if k >= max_batches:
            break
        y = s["label"].view(-1)
        counts += torch.bincount(y, minlength=num_classes).double()
    freq = counts / counts.sum().clamp_min(1.0)
    w = 1.0 / (freq + 1e-12)
    w = w / w.mean().clamp_min(1e-12)
    w = w.float()
    w = torch.clamp(w, clamp_min, clamp_max).to(device)
    return w, counts


def estimate_dice_weights_from_ce_weights(ce_w):
    w = ce_w.clone().detach().float()
    w[0] = min(float(w[0]), 1.0)
    w = w / w.mean().clamp_min(1e-12)
    return w


# ============================================================
#                 Test wrapper: volume -> 2D slices
# ============================================================

class TestVol2DSliceDataset(Dataset):
    def __init__(self, base_dataset):
        self.base = base_dataset
        self.index = []

        for case_id in range(len(self.base)):
            s = self.base[case_id]
            img = s["image"]
            if isinstance(img, np.ndarray):
                img = torch.from_numpy(img)
            if img.dim() == 4:
                D = img.shape[1]
            elif img.dim() == 3:
                D = img.shape[0]
            else:
                D = 1

            for z in range(D):
                self.index.append((case_id, z))

    def __len__(self):
        return len(self.index)

    def __getitem__(self, idx):
        case_id, z = self.index[idx]
        s = self.base[case_id]

        img = s["image"]
        lab = s["label"]

        if isinstance(img, np.ndarray):
            img = torch.from_numpy(img)
        if isinstance(lab, np.ndarray):
            lab = torch.from_numpy(lab)

        if img.dim() == 4:
            img2d = img[0, z]
        elif img.dim() == 3:
            img2d = img[z]
        elif img.dim() == 2:
            img2d = img
        else:
            raise RuntimeError(f"Unsupported image dim: {img.dim()}")

        if lab.dim() == 4:
            lab2d = lab[0, z]
        elif lab.dim() == 3:
            lab2d = lab[z]
        elif lab.dim() == 2:
            lab2d = lab
        else:
            raise RuntimeError(f"Unsupported label dim: {lab.dim()}")

        img2d = img2d.unsqueeze(0).float()
        lab2d = lab2d.long()

        return {"image": img2d, "label": lab2d}


# ============================================================
#                 2D prep
# ============================================================

def _prep_2d_batch(images, labels, img_size, device):
    if images.size(1) == 1:
        images = images.repeat(1, 3, 1, 1)

    if images.shape[-2] != img_size or images.shape[-1] != img_size:
        images = F.interpolate(images, size=(img_size, img_size), mode="bilinear", align_corners=False)

    if labels.shape[-2] != img_size or labels.shape[-1] != img_size:
        labels = labels.unsqueeze(1).float()
        labels = F.interpolate(labels, size=(img_size, img_size), mode="nearest")
        labels = labels.squeeze(1).long()

    imgs = images.to(device, dtype=torch.float32, non_blocking=True)
    gts = labels.to(device, dtype=torch.long, non_blocking=True)
    return imgs, gts


# ============================================================
#                 Attention U-Net
# ============================================================

class conv_block(nn.Module):
    def __init__(self, ch_in, ch_out):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)


class up_conv(nn.Module):
    def __init__(self, ch_in, ch_out):
        super().__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False),
            nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.up(x)


class Attention_block(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi


class AttU_Net(nn.Module):
    def __init__(self, img_ch=3, num_class=9):
        super().__init__()

        self.Maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(ch_in=img_ch, ch_out=64)
        self.Conv2 = conv_block(ch_in=64, ch_out=128)
        self.Conv3 = conv_block(ch_in=128, ch_out=256)
        self.Conv4 = conv_block(ch_in=256, ch_out=512)
        self.Conv5 = conv_block(ch_in=512, ch_out=1024)

        self.Up5 = up_conv(ch_in=1024, ch_out=512)
        self.Att5 = Attention_block(F_g=512, F_l=512, F_int=256)
        self.Up_conv5 = conv_block(ch_in=1024, ch_out=512)

        self.Up4 = up_conv(ch_in=512, ch_out=256)
        self.Att4 = Attention_block(F_g=256, F_l=256, F_int=128)
        self.Up_conv4 = conv_block(ch_in=512, ch_out=256)

        self.Up3 = up_conv(ch_in=256, ch_out=128)
        self.Att3 = Attention_block(F_g=128, F_l=128, F_int=64)
        self.Up_conv3 = conv_block(ch_in=256, ch_out=128)

        self.Up2 = up_conv(ch_in=128, ch_out=64)
        self.Att2 = Attention_block(F_g=64, F_l=64, F_int=32)
        self.Up_conv2 = conv_block(ch_in=128, ch_out=64)

        self.Conv_1x1 = nn.Conv2d(64, num_class, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x1 = self.Conv1(x)

        x2 = self.Maxpool(x1)
        x2 = self.Conv2(x2)

        x3 = self.Maxpool(x2)
        x3 = self.Conv3(x3)

        x4 = self.Maxpool(x3)
        x4 = self.Conv4(x4)

        x5 = self.Maxpool(x4)
        x5 = self.Conv5(x5)

        d5 = self.Up5(x5)
        x4_att = self.Att5(g=d5, x=x4)
        d5 = torch.cat((x4_att, d5), dim=1)
        d5 = self.Up_conv5(d5)

        d4 = self.Up4(d5)
        x3_att = self.Att4(g=d4, x=x3)
        d4 = torch.cat((x3_att, d4), dim=1)
        d4 = self.Up_conv4(d4)

        d3 = self.Up3(d4)
        x2_att = self.Att3(g=d3, x=x2)
        d3 = torch.cat((x2_att, d3), dim=1)
        d3 = self.Up_conv3(d3)

        d2 = self.Up2(d3)
        x1_att = self.Att2(g=d2, x=x1)
        d2 = torch.cat((x1_att, d2), dim=1)
        d2 = self.Up_conv2(d2)

        logits = self.Conv_1x1(d2)  # (B,C,H,W)
        return logits


# ============================================================
#                 TEST eval (strict 2D) + loss
# ============================================================

@torch.no_grad()
def evaluate_on_test_2d(model, test_loader, num_classes, img_size, device, ce_loss, dice_loss):
    model.eval()
    running = init_running_metrics(num_classes)

    loss_sum = 0.0
    ce_sum = 0.0
    dice_sum = 0.0
    n_batches = 0

    for samples in tqdm(test_loader, total=len(test_loader), desc="TEST_2D"):
        images = samples["image"]
        labels = samples["label"]

        imgs, gts = _prep_2d_batch(images, labels, img_size, device)

        logits = model(imgs)
        loss_ce = ce_loss(logits, gts)
        loss_d = dice_loss(logits, gts)
        loss = loss_ce + loss_d

        loss_sum += float(loss.detach().cpu())
        ce_sum += float(loss_ce.detach().cpu())
        dice_sum += float(loss_d.detach().cpu())
        n_batches += 1

        pred = torch.argmax(torch.softmax(logits, dim=1), dim=1)
        pred_np = pred.detach().cpu().numpy()
        gt_np = gts.detach().cpu().numpy()

        for b in range(pred_np.shape[0]):
            running = update_running_metrics(running, pred_np[b], gt_np[b], num_classes)

    per_class, macro = finalize_metrics(running, num_classes)
    test_loss = loss_sum / max(1, n_batches)
    test_ce = ce_sum / max(1, n_batches)
    test_dice_l = dice_sum / max(1, n_batches)

    return per_class, macro, (test_loss, test_ce, test_dice_l)


# ============================================================
#                 Args
# ============================================================

def get_argparser():
    p = argparse.ArgumentParser()

    p.add_argument("--versions_root", type=str, default="/root/.cache/kagglehub/datasets/dogcdt/synapse/versions/1")
    p.add_argument("--data_subdir", type=str, default="Synapse")
    p.add_argument("--list_dir", type=str, default="/root/lists_synapse")
    p.add_argument("--train_folder", type=str, default="train_npz")
    p.add_argument("--test_folder", type=str, default="test_vol_h5")

    p.add_argument("--num_classes", type=int, default=9)
    p.add_argument("--START_EPOCH", type=int, default=0)
    p.add_argument("--NB_EPOCH", type=int, default=50)

    p.add_argument("--LR", type=float, default=1e-4)
    p.add_argument("--weight_decay", type=float, default=1e-4)

    p.add_argument("--batch_size", type=int, default=4)
    p.add_argument("--test_batch_size", type=int, default=8)
    p.add_argument("--img_size", type=int, default=224)

    p.add_argument("--RESUME", type=bool, default=False)
    p.add_argument("--random_seed", type=int, default=1234)

    p.add_argument("--metrics_every", type=int, default=10)
    p.add_argument("--grad_clip", type=float, default=1.0)

    p.add_argument("--drive_root", type=str, default="/content/drive/MyDrive/Synapse_experiments")
    p.add_argument("--exp_name", type=str, default="Synapse_AttUNet_only")

    p.add_argument("--ce_weight_batches", type=int, default=200)
    p.add_argument("--ce_w_min", type=float, default=0.1)
    p.add_argument("--ce_w_max", type=float, default=10.0)

    p.add_argument("--save_best_on", type=str, default="macro_dice", choices=["macro_dice", "loss"])

    return p


# ============================================================
#                 Main
# ============================================================

def main():
    opts = get_argparser().parse_known_args()[0]

    drive.mount("/content/drive", force_remount=False)

    exp_dir = os.path.join(opts.drive_root, opts.exp_name)
    os.makedirs(exp_dir, exist_ok=True)
    ckpt_dir = os.path.join(exp_dir, "checkpoints")
    os.makedirs(ckpt_dir, exist_ok=True)

    log_path = setup_drive_logging(exp_dir)
    print("✅ Logging to:", log_path)
    print("✅ Exp dir:", exp_dir)

    csv_path = os.path.join(exp_dir, "history_attunet.csv")
    C = opts.num_classes

    header_cols = [
        "epoch", "lr",
        "train_loss", "train_ce", "train_dice_loss",
        "train_macro_dice", "train_macro_jac", "train_macro_hd95",
        "test_loss", "test_ce", "test_dice_loss",
        "test_macro_dice", "test_macro_jac", "test_macro_hd95",
    ]
    for c in range(1, C):
        header_cols += [
            f"train_dice_c{c:02d}", f"train_jac_c{c:02d}", f"train_hd95_c{c:02d}",
            f"test_dice_c{c:02d}",  f"test_jac_c{c:02d}",  f"test_hd95_c{c:02d}",
        ]
    header = ",".join(header_cols) + "\n"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device:", device)

    torch.manual_seed(opts.random_seed)
    np.random.seed(opts.random_seed)
    random.seed(opts.random_seed)

    tr_transform = RandomGenerator(output_size=[opts.img_size, opts.img_size])

    train_dataset = Synapse_dataset(
        versions_root=opts.versions_root,
        split="train",
        transform=tr_transform,
        list_dir=opts.list_dir,
        data_subdir=opts.data_subdir,
        train_folder=opts.train_folder,
        test_folder=opts.test_folder,
        verbose=True,
    )
    print("Train size:", len(train_dataset))

    train_loader = DataLoader(
        train_dataset,
        batch_size=opts.batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True,
    )

    test_base = Synapse_dataset(
        versions_root=opts.versions_root,
        split="test",
        transform=None,
        list_dir=opts.list_dir,
        data_subdir=opts.data_subdir,
        train_folder=opts.train_folder,
        test_folder=opts.test_folder,
        verbose=True,
    )
    print("Test cases:", len(test_base))

    test_dataset = TestVol2DSliceDataset(test_base)
    print("Test slices (2D):", len(test_dataset))

    test_loader = DataLoader(
        test_dataset,
        batch_size=opts.test_batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )

    model = AttU_Net(img_ch=3, num_class=C).to(device)

    last_path = os.path.join(ckpt_dir, "AttUNet_last.pth")
    best_path = os.path.join(ckpt_dir, "AttUNet_best.pth")

    if opts.RESUME and os.path.isfile(last_path):
        print("Loading AttUNet checkpoint:", last_path)
        model.load_state_dict(torch.load(last_path, map_location=device), strict=True)

    ce_w, counts = estimate_ce_weights_from_loader(
        train_loader, C, device,
        max_batches=opts.ce_weight_batches,
        clamp_min=opts.ce_w_min,
        clamp_max=opts.ce_w_max,
    )
    print("Pixel counts:", counts.cpu().numpy().astype(np.int64))
    print("CE weights:", ce_w.detach().cpu().numpy())

    ce_loss = nn.CrossEntropyLoss(weight=ce_w, reduction="mean")
    dice_w = estimate_dice_weights_from_ce_weights(ce_w)
    dice_loss = MultiClassDiceLoss(class_weights=dice_w, include_background=False)

    optimizer = torch.optim.AdamW(model.parameters(), lr=opts.LR, weight_decay=opts.weight_decay)

    best_score = -1e9 if opts.save_best_on == "macro_dice" else 1e9

    def _fmt(x):
        return "" if (x is None or (isinstance(x, float) and np.isnan(x))) else f"{x}"

    for epoch in range(opts.START_EPOCH, opts.NB_EPOCH):
        lr = optimizer.param_groups[0]["lr"]

        print("\n" + "=" * 70)
        print(f"Epoch {epoch}/{opts.NB_EPOCH-1} | lr={lr:.6g} | AttU-Net")
        print("=" * 70)

        model.train()
        running_train = init_running_metrics(C)

        loss_sum = 0.0
        ce_sum = 0.0
        dice_sum = 0.0
        n_batches = 0

        for i, samples in tqdm(enumerate(train_loader), total=len(train_loader), desc="TRAIN"):
            images = samples["image"]
            labels = samples["label"]

            if images.size(1) == 1:
                images = images.repeat(1, 3, 1, 1)

            imgs = images.to(device, dtype=torch.float32, non_blocking=True)
            gts = labels.to(device, dtype=torch.long, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)
            logits = model(imgs)

            loss_ce = ce_loss(logits, gts)
            loss_d = dice_loss(logits, gts)
            loss = loss_ce + loss_d

            loss.backward()
            if opts.grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), opts.grad_clip)
            optimizer.step()

            loss_sum += float(loss.detach().cpu())
            ce_sum += float(loss_ce.detach().cpu())
            dice_sum += float(loss_d.detach().cpu())
            n_batches += 1

            if i % 20 == 0:
                print(f"[batch {i:04d}] loss={loss.item():.4f} | ce={loss_ce.item():.4f} | dice={loss_d.item():.4f}")

            if opts.metrics_every > 0 and (i % opts.metrics_every == 0):
                with torch.no_grad():
                    pred = torch.argmax(torch.softmax(logits, dim=1), dim=1)
                    pred_np = pred.detach().cpu().numpy()
                    gt_np = gts.detach().cpu().numpy()
                    for b in range(pred_np.shape[0]):
                        running_train = update_running_metrics(running_train, pred_np[b], gt_np[b], C)

        train_loss = loss_sum / max(1, n_batches)
        train_ce = ce_sum / max(1, n_batches)
        train_dice_l = dice_sum / max(1, n_batches)

        train_per_class, train_macro = finalize_metrics(running_train, C)
        tr_macro_dice, tr_macro_jac, tr_macro_hd95 = train_macro

        print("\n--- Train Summary ---")
        print(f"[TRAIN] Loss={train_loss:.6f} | CE={train_ce:.6f} | DiceLoss={train_dice_l:.6f}")
        print(f"[TRAIN] Macro Dice={tr_macro_dice:.4f} | Jaccard={tr_macro_jac:.4f} | HD95={tr_macro_hd95:.4f}")

        test_per_class, test_macro, test_losses = evaluate_on_test_2d(
            model, test_loader, C, opts.img_size, device, ce_loss, dice_loss
        )
        te_macro_dice, te_macro_jac, te_macro_hd95 = test_macro
        test_loss, test_ce, test_dice_l = test_losses

        print("\n--- Test Summary (2D slices from test_vol) ---")
        print(f"[TEST] Loss={test_loss:.6f} | CE={test_ce:.6f} | DiceLoss={test_dice_l:.6f}")
        print(f"[TEST] Macro Dice={te_macro_dice:.4f} | Jaccard={te_macro_jac:.4f} | HD95={te_macro_hd95:.4f}")

        torch.save(model.state_dict(), last_path)
        print("✅ Saved AttUNet last checkpoint:", last_path)

        row = [
            str(epoch), f"{lr}",
            f"{train_loss}", f"{train_ce}", f"{train_dice_l}",
            _fmt(tr_macro_dice), _fmt(tr_macro_jac), _fmt(tr_macro_hd95),
            f"{test_loss}", f"{test_ce}", f"{test_dice_l}",
            _fmt(te_macro_dice), _fmt(te_macro_jac), _fmt(te_macro_hd95),
        ]
        for c in range(1, C):
            td, tj, th = train_per_class[c]
            vd, vj, vh = test_per_class[c]
            row += [_fmt(td), _fmt(tj), _fmt(th), _fmt(vd), _fmt(vj), _fmt(vh)]

        append_csv(csv_path, header, ",".join(row) + "\n")

        if opts.save_best_on == "macro_dice":
            score = te_macro_dice
            if (not np.isnan(score)) and score > best_score:
                best_score = score
                torch.save(model.state_dict(), best_path)
                print(f"✅ Saved AttUNet best checkpoint (test_macro_dice={best_score:.4f}):", best_path)
        else:
            score = train_loss
            if score < best_score:
                best_score = score
                torch.save(model.state_dict(), best_path)
                print(f"✅ Saved AttUNet best checkpoint (train_loss={best_score:.6f}):", best_path)

    torch.cuda.empty_cache()
    print("Done.")


if __name__ == "__main__":
    main()

TEST_2D:  61%|██████    | 119/196 [01:48<01:25,  1.11s/it]

# debugging